In [1]:
import os
import pandas as pd
from eodhd import APIClient
from datetime import datetime, timedelta
import requests
import datetime
import pytz


In [2]:
api_key = os.getenv('EODHD_API_KEY')
api_key

'67b5ff99b63306.32749083'

In [3]:
api = APIClient(api_key)
api

In [4]:
# -------------------------------
# Define the trading day time range
# -------------------------------

# US markets are listed in Eastern Time.
et = pytz.timezone("US/Eastern")

# To include pre-market (before the official 9:30 AM open), we start early (say, 7:00 AM ET).
# And we end at 4:00 PM ET, covering the entire regular session.
start_et = et.localize(datetime.datetime(2025, 2, 1, 7, 0, 0))
end_et   = et.localize(datetime.datetime(2025, 5, 15, 16, 0, 0))

# Convert both start and end times to UTC (the API requires Unix timestamps in UTC)
start_utc = start_et.astimezone(pytz.utc)
end_utc   = end_et.astimezone(pytz.utc)
from_timestamp = int(start_utc.timestamp())
to_timestamp   = int(end_utc.timestamp())

# Print the calculated times for verification (ISO format)
print("Requesting data from (UTC):", start_utc.isoformat())
print("Until (UTC):", end_utc.isoformat())


Requesting data from (UTC): 2025-02-01T12:00:00+00:00
Until (UTC): 2025-05-15T20:00:00+00:00


In [5]:
# -------------------------------
# Construct the API request URL
# -------------------------------
stock = 'GOOGL'
ticker = f'{stock}.US'          # For US tickers, only NYSE and NASDAQ are supported for 1 or 5-minutes data.
url = (
    f"https://eodhd.com/api/intraday/{ticker}"
    f"?interval=1m&api_token={api_key}&fmt=json"
    f"&from={from_timestamp}&to={to_timestamp}"
)


In [6]:
# -------------------------------
# Make the API request and decode the JSON
# -------------------------------
response = requests.get(url)
# Check for errors (optional but recommended)
if response.status_code != 200:
    print("Error fetching data:", response.status_code)
    print(response.text)
    exit()

data = response.json()

In [7]:
# -------------------------------
# Convert the response into a DataFrame and make the dates readable
# -------------------------------
df = pd.DataFrame(data)
df.to_csv(f'G:/My Drive/Ingegneria/Data Science GD/My-Practice/my models/Trading/Stock Analysis/Intraday stocks/{stock}.csv')
df

,timestamp,gmtoffset,datetime,open,high,low,close,volume
0,1738573200,0,2025-02-03 09:00:00,200.01000000,200.09000000,198.26000000,200.03000000,5135
1,1738573260,0,2025-02-03 09:01:00,200.08000000,200.09000000,199.86000000,199.90000000,2165
2,1738573320,0,2025-02-03 09:02:00,199.91000000,200.00000000,199.46000000,199.88000000,4996
3,1738573380,0,2025-02-03 09:03:00,199.77000000,200.09000000,199.75000000,199.84000000,5094
4,1738573440,0,2025-02-03 09:04:00,199.90000000,200.01000000,199.80000000,200.00000000,2940
...,...,...,...,...,...,...,...,...
68603,1747338960,0,2025-05-15 19:56:00,163.55000000,163.76000000,163.52000000,163.73000000,210190
68604,1747339020,0,2025-05-15 19:57:00,163.73000000,163.88000000,163.68000000,163.86500000,359239
68605,1747339080,0,2025-05-15 19:58:00,163.87000000,163.94000000,163.80000000,163.84500000,344882
68606,1747339140,0,2025-05-15 19:59:00,163.85000000,164.00000000,163.72500000,163.94000000,937597
